In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

### DANN training

-> Takes hdf5 train/test files generated with DANN_lumin_preproc.ipynb as input, and train a Domain Adversarial Neural Network

### Base modules

In [2]:
import pandas as pd
import numpy as np
import os
import h5py
import pickle

import matplotlib
import matplotlib.pyplot as plt

### Loading data

In [3]:
input_dir = '/data_CMS/cms/portales/HHresonant_DANNlumin/'

from lumin.nn.data.fold_yielder import FoldYielder
train_fy = FoldYielder(input_dir+'train.hdf5')
test_fy = FoldYielder(input_dir+'test.hdf5')

### Classifier definition

In [9]:
from lumin.nn.models.model_builder import ModelBuilder
from lumin.nn.models.model import Model
from lumin.nn.models.helpers import CatEmbedder

bs = 64 # ???
objective = 'classification'

### Network structure
model_args = {'body':{
                    'act': 'relu', # Hidden layer activation
                    'width': 64,   # nodes per layer
                    'depth': 4     # Number of layers
                     }
             }
### Optimiser settings
opt_args = {'opt':'adam', 'eps':1e-06}

### Category embedding (to check later)
cat_embedder=CatEmbedder.from_fy(train_fy) 

model_builder = ModelBuilder(objective,
                             cont_feats=train_fy.cont_feats,
                             n_out= 1,
                             cat_embedder=cat_embedder,
                             model_args = model_args,
                             opt_args=opt_args
                            )

Model(model_builder)

Inputs:
13 Continuous: ['mT1', 'bjet1_CvsL', 'bjet1_CvsB', 'tauH_SVFIT_mass', 'HHsvfit_deltaPhi', 'mT2', 'HHKin_chi2', 'ditau_deltaR', 'dau1_pt', 'tauHsvfitMet_deltaPhi', 'ditau_deltaR_per_tauH_MET_pt', 'bjet1_pt', 'HHKin_mass_raw']
                   
2  Categorical: ['dau1_decayMode', 'dau2_decayMode']
                   
0  Matrix elements: []
                   

Model:
<bound method Module.parameters of Sequential(
  (0): CatEmbHead(
    (embeds): ModuleList(
      (0): Embedding(5, 3)
      (1): Embedding(4, 2)
    )
  )
  (1): FullyConnected(
    (layers): Sequential(
      (0): Sequential(
        (0): Linear(in_features=18, out_features=64, bias=True)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
      )
      (3): Sequential(
        (0): Linear(in_features=64, out_features=6

### Learning rate optimizer

In [10]:
from lumin.optimisation.hyper_param import lr_find

lr_finder = lr_find(train_fy, model_builder, bs, lr_bounds=[1e-5,1e1])

ValueError: Using a target size (torch.Size([64, 2])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.